In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pickle


# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [2]:
ak = pd.read_csv('../datasets/alaska_single_engine_clean.csv', low_memory=False)

In [3]:
ak.shape

(5728, 35)

In [4]:
ak = ak.drop(['ntsb_no', 'mkey', 'n', 'probable_cause', 'event_time', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'aircraft_damage', 'event_type', 'report_type', 'has_safety_rec', 'highest_injury_level', 'is_accident', 'has_aircraft_damage'], axis=1)

In [5]:
categorical_cols = ak.select_dtypes(exclude=['number']).columns.tolist()

In [6]:
categorical_cols

['city',
 'make',
 'model',
 'aircraft_category',
 'airport_id',
 'airport_name',
 'scheduled',
 'purpose_of_flight',
 'far',
 'weather_condition',
 'operator',
 'event_season']

In [7]:
ak.columns

Index(['city', 'latitude', 'longitude', 'make', 'model', 'aircraft_category',
       'airport_id', 'airport_name', 'amateur_built', 'scheduled',
       'purpose_of_flight', 'far', 'weather_condition', 'operator',
       'event_year', 'event_month', 'event_day', 'event_hour', 'event_season',
       'has_injury'],
      dtype='object')

In [8]:
ak_dummies = pd.get_dummies(ak, columns = categorical_cols, drop_first=True)

In [9]:
X = ak_dummies.drop('has_injury', axis=1)
y = ak_dummies['has_injury']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.075, random_state=21)

### Baseline

In [11]:
1 - y.mean()

0.7122905027932961

### Benchmark Logistic Regression Model

In [12]:
pipe_log_reg_bench = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_bench', LogisticRegression(max_iter=1000))
])

In [13]:
pipe_log_reg_bench.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('log_reg_bench', LogisticRegression(max_iter=1000))])

In [14]:
pipe_log_reg_bench.score(X_train, y_train)

0.9596073990184976

In [15]:
pipe_log_reg_bench.score(X_test, y_test)

0.6697674418604651

### Tuned Logistic Regression Model

In [16]:
pipe_log_reg_tuned = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_tuned', LogisticRegression())
])

In [17]:
pipe_log_reg_tuned.get_params()

{'memory': None,
 'steps': [('sc', StandardScaler()), ('log_reg_tuned', LogisticRegression())],
 'verbose': False,
 'sc': StandardScaler(),
 'log_reg_tuned': LogisticRegression(),
 'sc__copy': True,
 'sc__with_mean': True,
 'sc__with_std': True,
 'log_reg_tuned__C': 1.0,
 'log_reg_tuned__class_weight': None,
 'log_reg_tuned__dual': False,
 'log_reg_tuned__fit_intercept': True,
 'log_reg_tuned__intercept_scaling': 1,
 'log_reg_tuned__l1_ratio': None,
 'log_reg_tuned__max_iter': 100,
 'log_reg_tuned__multi_class': 'auto',
 'log_reg_tuned__n_jobs': None,
 'log_reg_tuned__penalty': 'l2',
 'log_reg_tuned__random_state': None,
 'log_reg_tuned__solver': 'lbfgs',
 'log_reg_tuned__tol': 0.0001,
 'log_reg_tuned__verbose': 0,
 'log_reg_tuned__warm_start': False}

In [18]:
pipe_log_reg_tuned_params = {
    'log_reg_tuned__max_iter': [100, 1_000],
    'log_reg_tuned__C': np.linspace(0.00001, .01, 5),
    'log_reg_tuned__penalty': ['l1', 'l2']
}

In [19]:
gs= GridSearchCV(pipe_log_reg_tuned,
                                param_grid = pipe_log_reg_tuned_params,
                                cv = 5,
                               n_jobs=6)

In [20]:
%%time
gs.fit(X_train, y_train)

/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 420,

CPU times: user 4.54 s, sys: 1.17 s, total: 5.71 s
Wall time: 1min 16s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('sc', StandardScaler()),
                                       ('log_reg_tuned',
                                        LogisticRegression())]),
             n_jobs=6,
             param_grid={'log_reg_tuned__C': array([1.0000e-05, 2.5075e-03, 5.0050e-03, 7.5025e-03, 1.0000e-02]),
                         'log_reg_tuned__max_iter': [100, 1000],
                         'log_reg_tuned__penalty': ['l1', 'l2']})

In [21]:
gs.best_params_

{'log_reg_tuned__C': 1e-05,
 'log_reg_tuned__max_iter': 100,
 'log_reg_tuned__penalty': 'l2'}

In [22]:
gs.score(X_train, y_train)

0.7119667799169498

In [23]:
gs.score(X_test, y_test)

0.7162790697674418

In [24]:
y_pred = gs.predict(X_test)

In [25]:
confusion_matrix(y_test, y_pred)

array([[308,   0],
       [122,   0]])

In [26]:
print(classification_report(y_test, y_pred, digits = 5))

              precision    recall  f1-score   support

           0    0.71628   1.00000   0.83469       308
           1    0.00000   0.00000   0.00000       122

    accuracy                        0.71628       430
   macro avg    0.35814   0.50000   0.41734       430
weighted avg    0.51306   0.71628   0.59787       430



/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Coefficients

In [27]:
# coefficients = np.exp(gs.best_estimator_.named_steps['log_reg_tuned'].coef_[0]*.1)

In [28]:
coefficients = gs.best_estimator_.named_steps['log_reg_tuned'].coef_[0]

In [29]:
feature_names = list(X_train.columns)

In [30]:
coefficients_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefficients})

In [31]:
pd.set_option('display.max_rows', None)
coefficients_df.head(25)

,feature,coefficient
0,latitude,-0.000865
1,longitude,0.000813
2,amateur_built,0.001321
3,event_year,-0.001089
4,event_month,0.000130
5,event_day,0.000177
6,event_hour,-0.000811
7,city_100MI S.KNG SLM,-0.000210
8,city_11NM EAST OF SI,0.000504
9,city_18NM ESE KETCHI,-0.000201


In [32]:
coefficients_df.sort_values(by='coefficient', ascending=False).head(30)

,feature,coefficient
4132,airport_name_Unknown,0.002445
4228,weather_condition_Unknown,0.002371
4224,far_135,0.001974
3053,airport_id_Unknown,0.001959
4215,purpose_of_flight_UNK,0.001804
2,amateur_built,0.001321
2546,aircraft_category_HELI,0.001254
485,city_KETCHIKAN,0.001148
1242,make_De Havilland,0.001121
738,city_NOME,0.001093


In [33]:
coefficients_df.sort_values(by='coefficient', ascending=False).tail(30)

,feature,coefficient
3702,airport_name_MERRILL,-0.000677
1339,make_MAULE,-0.000691
2901,airport_id_ORT,-0.000696
2926,airport_id_PAFA,-0.000698
3009,airport_id_PVT,-0.000699
2982,airport_id_PATK,-0.000703
4206,purpose_of_flight_INST,-0.000709
3637,airport_name_LAKE HOOD STRIP,-0.000720
3746,airport_name_Merrill Field,-0.000724
2957,airport_id_PANC,-0.000725


In [34]:
coefficients_df.to_csv('../datasets/nolan_log_reg_coef.csv', index=False)

### Pickle

In [35]:
with open ('../pickles/nolan_log_reg.pkl', 'wb') as f:
    pickle.dump('gs.pkl', f)